## Part 1: Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [3]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [4]:
#Preprocess y - Encode the Binary output for Attrition (yes/no) with LabelEncoder and 
# Encode the multiple outputs for the Department feature with OneHotEncoder

# Preprocess "Department" column (one-hot encoding)
dept_encoder = OneHotEncoder(sparse_output=False)
dept_encoded = dept_encoder.fit_transform(attrition_df[['Department']])
dept_columns = dept_encoder.get_feature_names_out(['Department'])
df_dept_encoded = pd.DataFrame(dept_encoded, columns=dept_columns)

# Preprocess "Attrition" column (label encoding for binary; one-hot encoding for multiple categories)
attrition_encoder = OneHotEncoder(sparse_output=False)
attrition_encoded = attrition_encoder.fit_transform(attrition_df[['Attrition']])
attrition_columns = attrition_encoder.get_feature_names_out(['Attrition'])
df_attrition_encoded = pd.DataFrame(attrition_encoded, columns=attrition_columns)


# Concatenate the encoded columns to the original DataFrame
attrition_df_processed = pd.concat([df_attrition_encoded, df_dept_encoded], axis=1)
attrition_df_processed = pd.concat([attrition_df_processed, attrition_df], axis=1)

# Drop the original "Attrition" and "Department" columns
attrition_df_processed = attrition_df_processed.drop(['Attrition', 'Department'], axis=1)

attrition_df_processed.head()


,Attrition_No,Attrition_Yes,Department_Human Resources,Department_Research & Development,Department_Sales,Age,BusinessTravel,DistanceFromHome,Education,EducationField,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.0,1.0,0.0,0.0,1.0,41,Travel_Rarely,1,2,Life Sciences,...,3,1,0,8,0,1,6,4,0,5
1,1.0,0.0,0.0,1.0,0.0,49,Travel_Frequently,8,1,Life Sciences,...,4,4,1,10,3,3,10,7,1,7
2,0.0,1.0,0.0,1.0,0.0,37,Travel_Rarely,2,2,Other,...,3,2,0,7,3,3,0,0,0,0
3,1.0,0.0,0.0,1.0,0.0,33,Travel_Frequently,3,4,Life Sciences,...,3,3,0,8,3,3,8,7,3,0
4,1.0,0.0,0.0,1.0,0.0,27,Travel_Rarely,2,1,Medical,...,3,4,1,6,3,3,2,2,2,2


In [5]:
# Create y_df with the Attrition and Department columns

y_attrition=attrition_df_processed[['Attrition_No','Attrition_Yes']]
y_department=attrition_df_processed[['Department_Human Resources','Department_Research & Development','Department_Sales']]
print(y_attrition.head())
print(y_department.head())

   Attrition_No  Attrition_Yes
0           0.0            1.0
1           1.0            0.0
2           0.0            1.0
3           1.0            0.0
4           1.0            0.0
   Department_Human Resources  Department_Research & Development  \
0                         0.0                                0.0   
1                         0.0                                1.0   
2                         0.0                                1.0   
3                         0.0                                1.0   
4                         0.0                                1.0   

   Department_Sales  
0               1.0  
1               0.0  
2               0.0  
3               0.0  
4               0.0  


In [6]:
# Extracting the "OverTime" column for encoding
overtime_column = attrition_df_processed[['OverTime']]

# Initializing OneHotEncoder
ohe = OneHotEncoder(sparse_output=False, drop='first')  # 'drop="first"' to avoid multicollinearity

# Fitting and transforming the "OverTime" feature
overtime_encoded = ohe.fit_transform(overtime_column)

# Getting the encoded feature names
encoded_feature_names = ohe.get_feature_names_out(['OverTime'])

# Creating a DataFrame for the encoded features
overtime_encoded_df = pd.DataFrame(overtime_encoded, columns=encoded_feature_names)

# Concatenating the original dataframe with the encoded features
attrition_df_processed = pd.concat([attrition_df_processed, overtime_encoded_df], axis=1)

# Dropping the original "OverTime" column
attrition_df_processed.drop('OverTime', axis=1, inplace=True)





In [7]:
# Create a list of at least 10 column names to use as X data

features=['Education','Age','DistanceFromHome','JobSatisfaction','OverTime_Yes','StockOptionLevel','WorkLifeBalance','YearsAtCompany','YearsSinceLastPromotion','NumCompaniesWorked','TotalWorkingYears']

# Create X_df using your selected columns
X=attrition_df_processed[features]

# Show the data types for X_df

X.dtypes

Education                    int64
Age                          int64
DistanceFromHome             int64
JobSatisfaction              int64
OverTime_Yes               float64
StockOptionLevel             int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsSinceLastPromotion      int64
NumCompaniesWorked           int64
TotalWorkingYears            int64
dtype: object

In [8]:
# Displaying the column names in the dataframe
print(attrition_df_processed.columns)

Index(['Attrition_No', 'Attrition_Yes', 'Department_Human Resources',
       'Department_Research & Development', 'Department_Sales', 'Age',
       'BusinessTravel', 'DistanceFromHome', 'Education', 'EducationField',
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel',
       'JobRole', 'JobSatisfaction', 'MaritalStatus', 'NumCompaniesWorked',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager', 'OverTime_Yes'],
      dtype='object')


In [9]:
# Split data into training and testing sets

X_train, X_test, y_attrition_train, y_attrition_test, y_dept_train, y_dept_test = train_test_split(X, y_attrition, y_department)



In [10]:
# Create a StandardScaler
scaler=StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled=scaler.fit_transform(X_train)

# Scale the training and testing data
X_test_scaled=scaler.transform(X_test)


In [11]:
#Confirm Encoded y-Department structure
print(y_dept_test)
print(y_dept_train)

      Department_Human Resources  Department_Research & Development  \
212                          0.0                                0.0   
988                          0.0                                1.0   
75                           0.0                                1.0   
836                          0.0                                0.0   
742                          0.0                                1.0   
...                          ...                                ...   
1329                         1.0                                0.0   
798                          0.0                                1.0   
469                          0.0                                0.0   
562                          0.0                                1.0   
891                          0.0                                1.0   

      Department_Sales  
212                1.0  
988                0.0  
75                 0.0  
836                1.0  
742                0.0

In [12]:
#Confirm Encoded y-Attrition (yes) structure
print(y_attrition_test)
print(y_attrition_train)

      Attrition_No  Attrition_Yes
212            1.0            0.0
988            1.0            0.0
75             1.0            0.0
836            0.0            1.0
742            1.0            0.0
...            ...            ...
1329           1.0            0.0
798            0.0            1.0
469            0.0            1.0
562            0.0            1.0
891            1.0            0.0

[368 rows x 2 columns]
      Attrition_No  Attrition_Yes
1293           1.0            0.0
982            1.0            0.0
1021           0.0            1.0
957            1.0            0.0
721            1.0            0.0
...            ...            ...
1000           1.0            0.0
313            1.0            0.0
501            1.0            0.0
744            0.0            1.0
88             1.0            0.0

[1102 rows x 2 columns]


## Create, Compile, and Train the Model

In [13]:
# Find the number of columns in the X training data

print(X_train.shape[1])

# Create the input layer
input_layer = layers.Input(shape=(X_train.shape[1],), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu')(shared_layer1)



11


In [14]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_dense = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
dept_output = layers.Dense(len(y_dept_train.columns),
                             activation='softmax',
                             name='dept_output')(dept_dense)


In [15]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_dense = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(len(y_attrition_train.columns),
                             activation='sigmoid',
                             name='attrition_output')(attrition_dense)


In [16]:
# Create the model

model = Model(inputs=input_layer, outputs=[attrition_output, dept_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'dept_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'dept_output': 'accuracy', 'attrition_output': 'accuracy'})


# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │        768 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │      8,320 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      4,128 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      4,128 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_output (Dense) │ (None, 3)         │         99 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,509 (68.39 KB)

 Trainable params: 17,509 (68.39 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Train the model

model.fit(
    X,
    {'dept_output': y_department, 'attrition_output': y_attrition},
    epochs=100,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - attrition_output_accuracy: 0.8267 - dept_output_accuracy: 0.5748 - loss: 1.8266 - val_attrition_output_accuracy: 0.8537 - val_dept_output_accuracy: 0.6020 - val_loss: 1.2779
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step - attrition_output_accuracy: 0.8268 - dept_output_accuracy: 0.6570 - loss: 1.2355 - val_attrition_output_accuracy: 0.8537 - val_dept_output_accuracy: 0.6020 - val_loss: 1.2731
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 683us/step - attrition_output_accuracy: 0.8363 - dept_output_accuracy: 0.6447 - loss: 1.1933 - val_attrition_output_accuracy: 0.8537 - val_dept_output_accuracy: 0.5986 - val_loss: 1.2821
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - attrition_output_accuracy: 0.8405 - dept_output_accuracy: 0.6389 - loss: 1.1918 - val_attrition_output_accuracy: 0.8537 - val_dept_output_accuracy: 0.5952 - val_loss: 1.2551
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - attrition_output_accuracy: 0.83

In [18]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test, {'dept_output': y_dept_test, 'attrition_output': y_attrition_test})
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 378us/step - attrition_output_accuracy: 0.9087 - dept_output_accuracy: 0.7336 - loss: 0.9056


[0.9286691546440125, 0.8994565010070801, 0.7038043737411499]

In [19]:
# Print the accuracy for both department and attrition
# Print the quality and color accuracy
print(f"Department Accuracy: {test_results[2]}")
print(f"Attrition Accuracy: {test_results[1]}")


Department Accuracy: 0.7038043737411499
Attrition Accuracy: 0.8994565010070801


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.  Collectively, precision, recall, and the F1 score should be used as a whole, especially if the cost of false positives and false negatives are different.  If the model is being used to introduce intravention to mitigate attrition, the risk of a false positive is proably low.
2.  I choose the softmax for the department because there were multiple departments that coudl have been selected.  For the Attrition, I selected a sigmoid because the prediction of attrition was either yes or no, which sigmoind is best suited.
3. 	a.	Increase Dataset Size - Collect more data to provide the models with a more representative sample of the problem space.
	b.	Cross-Validation - Use cross-validation techniques to get a better estimate of the model’s performance and avoid overfitting.
	c.	Hyperparameter Tuning - Perform hyperparameter tuning on Random Forest and Gradient Boosting models to further improve performance.
	d.	Feature Engineering:  Explore creating new features or including more features into the model.